# Rf 7 0 1_Efficiencyfit
Speecial p.d.f.'s: unbinned maximum likelihood fit of an efficiency eff(x) function

to a dataset D(x,cut), where cut is a category encoding a selection, of which the efficiency as function of x should
be described by eff(x)



**Author:** 07/2008 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 03:00 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooGaussian.h"
#include "RooConstVar.h"
#include "RooFormulaVar.h"
#include "RooProdPdf.h"
#include "RooEfficiency.h"
#include "RooPolynomial.h"
#include "RooCategory.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "RooPlot.h"

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

Construct efficiency function e(x)
 -------------------------------------------------------------------

Declare variables x,mean,sigma with associated name, title, initial value and allowed range

In [3]:
RooRealVar x("x", "x", -10, 10);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Efficiency function eff(x;a,b)

In [4]:
RooRealVar a("a", "a", 0.4, 0, 1);
RooRealVar b("b", "b", 5);
RooRealVar c("c", "c", -1, -10, 10);
RooFormulaVar effFunc("effFunc", "(1-a)+a*cos((x-c)/b)", RooArgList(a, b, c, x));

Construct conditional  efficiency pdf e(cut|x)
 ------------------------------------------------------------------------------------------

Acceptance state cut (1 or 0)

In [5]:
RooCategory cut("cut", "cutr");
cut.defineType("accept", 1);
cut.defineType("reject", 0);

Construct efficiency p.d.f eff(cut|x)

In [6]:
RooEfficiency effPdf("effPdf", "effPdf", effFunc, cut, "accept");

Generate data (x, cut) from a toy model
 -----------------------------------------------------------------------------

Construct global shape p.d.f shape(x) and product model(x,cut) = eff(cut|x)*shape(x)
 (These are _only_ needed to generate some toy MC here to be used later)

In [7]:
RooPolynomial shapePdf("shapePdf", "shapePdf", x, RooConst(-0.095));
RooProdPdf model("model", "model", shapePdf, Conditional(effPdf, cut));

Generate some toy data from model

In [8]:
RooDataSet *data = model.generate(RooArgSet(x, cut), 10000);

Fit conditional efficiency pdf to data
 --------------------------------------------------------------------------

Fit conditional efficiency p.d.f to data

In [9]:
effPdf.fitTo(*data, ConditionalObservables(x));

[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 a            4.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
     2 c           -1.00000e+00  2.00000e+00   -1.00000e+01  1.00000e+01
 **********
 **    3 **SET ERR         0.5
 **********
 **********
 **    4 **SET PRINT           1
 **********
 **********
 **    5 **SET STR           1
 **********
 NOW USING STRATEGY  1: TRY TO BALANCE SPEED AGAINST RELIABILITY
 **********
 **    6 **MIGRAD        1000           1
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
 START MIGRAD MINIMIZATION.  STRATEGY  1.  CONVERGENCE WHEN EDM .LT. 1.00e-03
 FCN=3887.11 FROM MIGRAD    STATUS=INITIATE        8 CALLS           9 TOTAL
                     EDM= unknown      STRATEGY= 1      NO ERROR MATRIX 

Plot fitted, data efficiency
 --------------------------------------------------------

Plot distribution of all events and accepted fraction of events on frame

In [10]:
RooPlot *frame1 = x.frame(Bins(20), Title("Data (all, accepted)"));
data->plotOn(frame1);
data->plotOn(frame1, Cut("cut==cut::accept"), MarkerColor(kRed), LineColor(kRed));

[#1] INFO:Plotting -- RooTreeData::plotOn: plotting 8176 events out of 10000 total events


Plot accept/reject efficiency on data overlay fitted efficiency curve

In [11]:
RooPlot *frame2 = x.frame(Bins(20), Title("Fitted efficiency"));
data->plotOn(frame2, Efficiency(cut)); // needs ROOT version >= 5.21
effFunc.plotOn(frame2, LineColor(kRed));

Draw all frames on a canvas

In [12]:
TCanvas *ca = new TCanvas("rf701_efficiency", "rf701_efficiency", 800, 400);
ca->Divide(2);
ca->cd(1);
gPad->SetLeftMargin(0.15);
frame1->GetYaxis()->SetTitleOffset(1.6);
frame1->Draw();
ca->cd(2);
gPad->SetLeftMargin(0.15);
frame2->GetYaxis()->SetTitleOffset(1.4);
frame2->Draw();

Draw all canvases 

In [13]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()